# 2. domača naloga

In [3]:
import pandas as pd

## 1.naloga

In [4]:
filmi = pd.read_csv('filmi.csv')
filmi

,naslov,leto,oskar,stevilo
0,12 Years a Slave,2013,False,6
1,12 Years a Slave,2013,True,3
2,1917,2019,False,7
3,1917,2019,True,3
4,20 Days in Mariupol,2023,False,0
...,...,...,...,...
2713,Zootopia,2016,True,1
2714,Zorba the Greek,1964,False,4
2715,Zorba the Greek,1964,True,3
2716,tom thumb,1958,False,0


(a) Kateri film (oz.filmi, če jih je več) je prejel največ nominacij ter koliko?

In [54]:
zdruzeno_po_filmih = filmi.groupby('naslov')['stevilo'].sum()
najvec = max(zdruzeno_po_filmih)
zdruzeno_po_filmih = zdruzeno_po_filmih.reset_index()
odgovor1a = zdruzeno_po_filmih[zdruzeno_po_filmih['stevilo']==najvec]
print(odgovor1a)

            naslov  stevilo
40  A Star Is Born       19


(b) Število filmov, ki so bili nominirani vsaj trikrat in so osvojili oskarja za vsaj polovico nominacij.

In [8]:
zdruzeni_po_filmih = filmi.groupby('naslov')
ustrezni_filmi = zdruzeni_po_filmih.filter(lambda group: (group['stevilo'].sum()>=3 and group.loc[group['oskar']==True,'stevilo'].sum()>=0.5*group['stevilo'].sum()))
odgovor1b = len(ustrezni_filmi['naslov'].unique())
print(odgovor1b)

150


Desetletje in stoletje, v katerem je bilo nagrajenih največ različnih filmov ter koliko.

In [9]:
filmi['desetletje'] = (filmi['leto']//10 + 1) # Npr. 1985 → 1980
filmi['stoletje'] = (filmi['leto'] // 100) + 1 # Npr. 1985 → 20. stoletje
# Skupno število unikatnih filmov po desetletjih
# Obdelujemo le filme, ki so bili nagrajeni z oskarjem (vsebuje True)
# Združimo jih po desetletju
# Za vsako desetletje preštejemo št različnih filmov nagrajenih v tem desetletju
desetletje_counts = filmi[filmi['oskar']].groupby('desetletje')['naslov'].nunique()
# Skupno število unikatnih filmov po stoletjih
# Enako kot prej, le da po stoletjih
stoletje_counts = filmi[filmi['oskar']].groupby('stoletje')['naslov'].nunique()
# Največje število filmov po desetletjih in stoletjih
najvecje_desetletje = desetletje_counts.idxmax(), desetletje_counts.max()
najvecje_stoletje = stoletje_counts.idxmax(), stoletje_counts.max()
print(f"Največ filmov nagrajenih v desetletju: V {najvecje_desetletje[0]} desetletju, in sicer {najvecje_desetletje[1]}")
print(f"Največ filmov nagrajenih v stoletju: V {najvecje_stoletje[0]} stoletju, in sicer {najvecje_stoletje[1]}")

Največ filmov nagrajenih v desetletju: V 195 desetletju, in sicer 174
Največ filmov nagrajenih v stoletju: V 20 stoletju, in sicer 1000


Nekateri naslovi filmov se ponavljajo. Kateri izmed filmov, katerih naslovi se ponovijo natanko dvakrat, imajo tudi enako število prejetih oskarjev?

In [22]:
# 1. Filtriramo samo vrstice, kjer je 'oskar' True
df_filtered = filmi[filmi['oskar'] == True]

# 2. Združimo po naslovu filma, izračunamo število oskarjev za vsak film in število pojavitev
df_grouped = (
    df_filtered.groupby('naslov')
    .agg(oskar_sum=('oskar', 'sum'), num_occurences=('naslov', 'size'))
    .reset_index()
)

# 3. Preverimo, kateri filmi se pojavijo natanko 2-krat
df_grouped = df_grouped[df_grouped['num_occurences'] == 2]

# 4. Preverimo, kateri filmi imajo enako število prejetih oskarjev za obe vrstici
# Če so oskar_sum vrednosti za film enake, potem je število oskarjev enako
odgovor1d = df_grouped[df_grouped['oskar_sum'].duplicated(keep=False)]
odgovor1d['naslov']

63      All Quiet on the Western Front
233                          Cleopatra
263                 Cyrano de Bergerac
434                            Henry V
516                          King Kong
986                            The Fly
1010                  The Great Gatsby
1098           The Old Man and the Sea
1231                           Titanic
1270                                Up
1291                   West Side Story
Name: naslov, dtype: object

## 2. naloga

In [5]:
tuji_filmi = pd.read_csv('tuji_filmi.csv')
populacija = pd.read_csv('populacija.csv')

(a)  Število vseh nominacij in število vseh dobljenih oskarjev (tudi oskar za najboljši mednarodni celovečerni film) glede na leto za države iz tabele tuji filmi z izjemo ZDA. Rezultat predstavi s tabelo, ki ima štiri stolpce: drzava, leto, st nominacij in st oskarjev. Tabelo shrani v datoteko tabela2a.csv.
Namig: v tabeli filmi so upoštevani vsi oskarji, v tabeli tuji filmi pa vsi, razen oskarja za najboljši mednarodni celovečerni film.

In [12]:
# Filtriramo tuje filme brez ZDA
tuji_filmi = tuji_filmi[tuji_filmi["drzava"] != "ZDA"]

# Združimo tabele tuji_filmi in filmi (glede na naslov filma in leto)
zdruzeno = pd.merge(
    tuji_filmi,
    filmi,
    left_on=["naslov_nom", "leto"],
    right_on=["naslov", "leto"],
    how="left"
)

# Preverimo, ali je bil oskar dobljen (če je kategorija prisotna v tabeli `filmi`)
zdruzeno["oskar_dobljen"] = zdruzeno["kategorija"].notna()

# Štetje nominacij glede na državo in leto
nominacije = (
    tuji_filmi.groupby(["drzava", "leto"])
    .size()
    .reset_index(name="st_nominacij")
)

# Štetje dobljenih oskarjev glede na državo in leto
oskarji = (
    zdruzeno[zdruzeno["oskar_dobljen"] == True]
    .groupby(["drzava", "leto"])
    .size()
    .reset_index(name="st_oskarjev")
)

# Združimo podatke o nominacijah in oskarjih
rezultat = pd.merge(nominacije, oskarji, on=["drzava", "leto"], how="left")
rezultat["st_oskarjev"] = rezultat["st_oskarjev"].fillna(0).astype(int)

# Shrani rezultat v CSV datoteko
rezultat.to_csv("tabela2a.csv", index=False)


(b) Največ koliko nominacij je prejela država na eni podelitvi? Rezultat shrani v spremenljivko odgovor2b

In [14]:
# Združimo podatke glede na državo in leto ter preštejemo nominacije
nominacije = (tuji_filmi.groupby(["drzava", "leto"]).size().reset_index(name="st_nominacij"))

# Poiščemo največje število nominacij na eni podelitvi
odgovor2b = nominacije["st_nominacij"].max()
print(odgovor2b)

12


(c) Največ koliko oskarjev je prejela država na eni podelitvi? Rezultat shrani v spremenljivko odgovor2c.

In [16]:

# Združimo tabele tuji_filmi in filmi (glede na naslov filma in leto)
zdruzeno = pd.merge(
    tuji_filmi,
    filmi,
    left_on=["naslov_nom", "leto"],
    right_on=["naslov", "leto"],
    how="left"
)

# Filtriramo oskarje (film mora imeti kategorijo, kar pomeni, da je dobil oskarja)
dobljeni_oskarji = zdruzeno[zdruzeno["kategorija"].notna()]

# Združimo dobljene oskarje po državi in letu ter jih preštejemo
oskarji_stevilo = (
    dobljeni_oskarji.groupby(["drzava", "leto"])
    .size()
    .reset_index(name="st_oskarjev")
)

# Poiščemo največje število oskarjev na eni podelitvi
odgovor2c = oskarji_stevilo["st_oskarjev"].max()
print(odgovor2c)

12


(d)  Kateri film je prejel oskarje iz prejšnje točke? Rezultat shrani v spremenljivko odgovor2d

In [17]:
# Filtriramo države in leta, kjer so oskarji dosegli največje število
maksimalni_oskarji = oskarji_stevilo[oskarji_stevilo["st_oskarjev"] == odgovor2c]

# Poiščemo naslove filmov z največ oskarji
filmi_z_max_oskarji = zdruzeno[
    (zdruzeno["drzava"].isin(maksimalni_oskarji["drzava"])) &
    (zdruzeno["leto"].isin(maksimalni_oskarji["leto"]))
]

# Izluščimo unikatne naslove filmov, ki ustrezajo pogojem
odgovor2d = filmi_z_max_oskarji["naslov_nom"].unique()

print(odgovor2d)

['All Quiet on the Western Front' 'Fanny and Alexander'
 'Life Is Beautiful']


(e)  Za države iz tabele tuji filmi z izjemo ZDA izračunaj delež uspešnih nominacij (nominacija je uspešna, če film prejme oskarja). Rezultat predstavi s tabelo, ki ima dva stolpca: drzava in delez. Tabelo shrani v datoteko tabela2e.csv. Katera država ima najmanjši delež uspešnih nominacij? Rezultat shrani v spremenljivko odgovor2e.

In [36]:
# Združimo tabelo 'tuji_filmi' s tabelo 'filmi' glede na 'naslov_nom' in 'leto'
novi_tuji_filmi = tuji_filmi.merge(
    filmi[['naslov', 'leto', 'oskar']],  # Uporabimo pravilne stolpce iz tabele 'filmi'
    left_on=['naslov_nom', 'leto'],  # Uporabimo ustrezne stolpce za združevanje
    right_on=['naslov', 'leto'],
    how='left',  # Ohranimo vse vrstice iz 'tuji_filmi'
    suffixes=('_tuji', '_filmi')  # Dodamo različne končnice za stolpce, ki se ponavljajo
)

# Odstranimo vrstice za ZDA
novi_tuji_filmi = novi_tuji_filmi[novi_tuji_filmi['drzava'] != 'ZDA']

# Dodamo stolpec 'oskar', ki označuje uspešne nominacije
novi_tuji_filmi['oskar'] = novi_tuji_filmi['oskar_filmi'] == True

# Izračunamo število nominacij in oskarjev za vsako državo
agg_data = novi_tuji_filmi.groupby('drzava').agg(
    nominacije=('naslov_nom', 'count'),  # Število nominacij za vsako državo
    oskarji=('oskar', 'sum')  # Število oskarjev (True pomeni uspešno nominacijo)
).reset_index()

# Izračunamo delež uspešnih nominacij
agg_data['delez'] = agg_data['oskarji'] / agg_data['nominacije']

# Rezultate shranimo v CSV datoteko
agg_data[['drzava', 'delez']].to_csv("tabela2e.csv", index=False)

# Poiščemo državo z najmanjšim deležem uspešnih nominacij
najmanjsi_delez = agg_data.loc[agg_data['delez'].idxmin()]
odgovor2e = najmanjsi_delez['drzava']

# Izpišemo državo z najmanjšim deležem
print(odgovor2e)

Czech Republic


(f) Število nominacij na milijon prebivalcev. Rezultat predstavi s tabelo, ki ima dva stolpca: drzava in nominacije na mio. Uporabi podatke o populaciji iz leta 2015, če podatka ni, naj države ne bo v končni tabeli. Tabelo shrani v datoteko tabela2f.csv

In [22]:
populacija = pd.read_csv('populacija.csv')

# Filtriramo podatke o populaciji za leto 2015 in spremenljivko "Capital city population (thousands)"
populacija_2015 = populacija[(populacija["leto"] == 2015) & (populacija["spremenljivka"] == "Capital city population (thousands)")]
populacija_2015.loc[:,'vrednost'] = populacija_2015['vrednost']/1000
populacija_2015 = populacija_2015[["enota", "vrednost"]].rename(columns={"enota": "drzava", "vrednost": "populacija_v_miljonih"})


# Združimo nominacije po državah
nominacije_po_drzavah = tuji_filmi.groupby("drzava").size().reset_index(name="nominacije")

# Združimo podatke o nominacijah in populaciji
zdruzeni_podatki = pd.merge(nominacije_po_drzavah, populacija_2015, on="drzava", how="inner")

# Izračunamo nominacije na tisoč prebivalcev glavnega mesta
zdruzeni_podatki["nominacije_na_miljon"] = zdruzeni_podatki["nominacije"] / zdruzeni_podatki["populacija_v_miljonih"]

# Končna tabela z dvema stolpcema
rezultat = zdruzeni_podatki[["drzava", "nominacije_na_miljon"]]

# Shranimo rezultat v datoteko
rezultat.to_csv("tabela2f.csv", index=False)
rezultat

,drzava,nominacije_na_miljon
0,Algeria,0.385802
1,Argentina,0.135999
2,Brazil,0.479846
3,Chile,0.306607
4,China,0.162858
5,France,1.304267
6,Germany,2.561184
7,Greece,0.316356
8,India,0.077322
9,Italy,4.133236


(g)  Kateri filmi iz tabele tuji filmi so prejeli tudi oskarja za najboljši mednarodni celovečerni film? Seznam filmov shrani v spremenljivko odgovor2g.

In [23]:
# Preverimo, kateri naslovi iz tabele "tuji_filmi" so prisotni v tabeli "filmi"
naslovi_tujih = set(tuji_filmi["naslov_nom"])
naslovi_glavni = set(filmi["naslov"])

# Naslovi, ki so prisotni v obeh tabelah
prisotni_v_glavni = naslovi_tujih & naslovi_glavni

# Filtriramo tuje filme, ki so prisotni v glavni tabeli "filmi"
tuji_filmi_prisotni = tuji_filmi[tuji_filmi["naslov_nom"].isin(prisotni_v_glavni)]

# Združimo z glavno tabelo "filmi" za nadaljnjo analizo
oskar_nagrajeni_prisotni = pd.merge(tuji_filmi_prisotni, filmi, left_on="naslov_nom", right_on="naslov")

# Filtriramo samo tiste filme, ki so prejeli oskarja
oskar_nagrajeni_prisotni = oskar_nagrajeni_prisotni[oskar_nagrajeni_prisotni["oskar"] == True]

# Izluščimo seznam naslovov, ki so prejeli oskarja
odgovor2g = oskar_nagrajeni_prisotni["naslov_nom"].unique()

print(odgovor2g)

Filmi, ki so prejeli oskarja za najboljši mednarodni celovečerni film:
['20 Days in Mariupol' 'A Man and a Woman'
 'All Quiet on the Western Front' 'Anatomy of a Fall' 'Cries and Whispers'
 'Crouching Tiger, Hidden Dragon' 'Cyrano de Bergerac'
 'Divorce Italian Style' 'Gate of Hell' 'Godzilla Minus One'
 'La Dolce Vita' 'Letters from Iwo Jima' 'Life Is Beautiful'
 'Marie-Louise' 'Never on Sunday' "Pan's Labyrinth" 'Parasite' 'RRR' 'Ran'
 'Roma' 'Serengeti Shall Not Die' 'Talk to Her' 'The Artist'
 'The Boy and the Heron' 'The Motorcycle Diaries' 'The Red Balloon'
 'The Zone of Interest' 'Z']


## 3.naloga

In [74]:
from plotnine import (
    ggplot, aes, geom_line, geom_point, labs, theme, element_text, scale_x_continuous
)

(a) Podeljeni oskarji

In [77]:
filmi = filmi[filmi["oskar"] == True]

# Štejemo število podeljenih oskarjev na leto
oskarji_po_letih = filmi.groupby('leto')['stevilo'].sum().reset_index(name='Število oskarjev')

# Poskrbimo, da so vključena samo leta, za katera obstajajo podatki
oskarji_po_letih = oskarji_po_letih.sort_values(by='leto').reset_index(drop=True)

# Preimenujemo stolpce za boljši pregled
oskarji_po_letih = oskarji_po_letih.rename(columns={"leto": "Leto"})

# Podeljeni oskarji
plot_oskarji = (
    ggplot(oskarji_po_letih, aes(x="Leto", y="Število oskarjev")) +
    geom_line(color="darkred") +
    geom_point(color="darkred") +
    labs(
        title="Oskarji",
        subtitle="Število podeljenih oskarjev med leti 1927 in 2022",
        x="Leto",
        y="Število podeljenih oskarjev"
    ) +
    scale_x_continuous(
        breaks=range(1927,2023,5)
    ) +
    theme(
        axis_text_x=element_text(rotation=45, hjust=1),
        figure_size=(7, 5),
        plot_title=element_text(hjust=0.5),  # Sredinska poravnava glavnega naslova
        plot_subtitle=element_text(hjust=0.5),  # Sredinska poravnava podnaslova
        axis_title_x=element_text(color="darkred"),  # Rdeč naslov osi x
        axis_title_y=element_text(color="darkred"),  # Rdeč naslov
    )
)


plot_oskarji.save("graf3a.pdf")

c:\Users\Aneja\Documents\pavp\venv\Lib\site-packages\plotnine\ggplot.py:615: PlotnineWarning: Saving 7 x 5 in image.
c:\Users\Aneja\Documents\pavp\venv\Lib\site-packages\plotnine\ggplot.py:616: PlotnineWarning: Filename: graf3a.pdf


(b) Prebivalstvo v glavnem mestu